# Decode line codes in png graphs

## Assumptions (format):
* The clock is given and it is a red line on the top.
* The signal line is black
* ...

In [1]:
# Makes sure to install PyPNG image handling module
import sys
!{sys.executable} -m pip install pypng

In [2]:
import png

In [176]:
r = png.Reader("ex.png")
t = r.asRGB()

img = list(t[2])
# print(img)

## Outline
The outline of the idea is:

1. Find the red lines that represent parallel synchronization signal above
2. Calculate their size
3. "Synchromize with rows below" (according to the rules of the code)
4. ...
5. PROFIT!


## !!! Things to keep in mind:
1. deviations of red
2. deviations of black
3. noise - it might just break everything!

A rather simple PNG we'll work with first:

![PNG example - NRZ](ex.png)

In [141]:
# Let us first define colour red
# We'll work with RGB for colours
# So for accepted variants we'll make a list of 3-lists.

class colourlist(list):
    """Just lists of 3-lists with some fancy methods to work with RGB colours
    """
    
    def add_deviations(self, d=8): # Magical numbers are so magical!
        """Adds deviations for RGB colours to a given list.
           Warning! Too huge - it takes forever.
           Input: list of 3-lists
           Output: None (side-effects - changes the list)
        """
    
        #l = l[:] Nah, let's make it a method
        l = self
    
        v = len(l)
        max_deviation = d
    
        for i in range(v): # Iterate through the list of colours
        
            for j in range(-max_deviation, max_deviation+1): # Actually it is the deviation.
                
                #for k in range(3): # RGB! (no "a"s here)
                    
                newcolour = self[i][:] # Take one of the original colours
                newcolour[0] = abs(newcolour[0]+j) # Create a deviation
                l.append(newcolour) # Append new colour to the end of the list. 
                # <- Here it is changed!
                for j in range(-max_deviation, max_deviation+1): # Work with all the possibilities with this d
                    newcolour1 = newcolour[:]
                    newcolour1[1] = abs(newcolour1[1]+j)
                    l.append(newcolour1) # Append new colour to the end of the list. Yeah! 
                    # <- Here it is changed!
                
                    for j in range(-max_deviation, max_deviation+1): # Work with all the possibilities with this d
                        newcolour2 = newcolour1[:]
                        newcolour2[2] = abs(newcolour2[2]+j)
                        l.append(newcolour2) # Append new colour to the end of the list. Yeah! 
                        # <- Here it is changed!
    
        return None

def withinDeviation(colour, cl, d=20):
    """This is much more efficient!
       Input: 3-list (colour), colourlist, int
       Output: bool
    """
    for el in cl:
        if (abs(colour[0] - el[0]) <= d and abs(colour[1] - el[1]) <= d and abs(colour[2] - el[2]) <= d):
            return True
    return False



accepted_colours = colourlist([[118, 58, 57], [97, 71, 36], [132, 56, 46], [132, 46, 47], [141, 51, 53]]) # ...

#accepted_colours.add_deviations()

# print(accepted_colours) # -check it! - or better don't - it is a biiiig list....

# print(len(accepted_colours)) # That will take a while... Heh..

In [142]:
def find_first_pixel_of_colour(pixellist, accepted_deviations):
    """Returns the row and column of the pixel 
       in a converted to list with RGB colours PNG
       Input: ..., colourlist
       Output: 2-tuple of int (or None)
    """
    
    accepted_deviations = accepted_deviations[:]
    rows = len(pixellist)
    cols = len(pixellist[0])
    
    for j in range(rows):
        for i in range(0, cols, 3):
            # if [pixellist[j][i], pixellist[j][i+1], pixellist[j][i+2]] in accepted_deviations:
            if withinDeviation([pixellist[j][i], pixellist[j][i+1], pixellist[j][i+2]], accepted_deviations):
                return (j, i)
    
    return None



fr = find_first_pixel_of_colour(img, accepted_colours)

if fr is None:
    print("Warning a corrupt file or a wrong format!!!")

print(fr)
print(img[fr[0]][fr[1]], img[fr[0]][fr[1]+1], img[fr[0]][fr[1]+2])
print(img[fr[0]])

(1, 3)
123 54 57
array('B', [255, 242, 244, 123, 54, 57, 141, 51, 53, 138, 44, 42, 114, 67, 39, 123, 171, 95, 97, 193, 95, 91, 186, 94, 126, 172, 107, 116, 68, 48, 141, 45, 49, 142, 43, 46, 116, 68, 46, 122, 172, 99, 93, 190, 93, 90, 187, 90, 130, 181, 106, 113, 66, 38, 141, 47, 45, 147, 52, 56, 109, 62, 42, 128, 176, 102, 89, 185, 87, 98, 193, 101, 126, 172, 107, 112, 64, 44, 139, 43, 47, 146, 47, 50, 114, 66, 44, 125, 175, 102, 91, 188, 91, 95, 192, 95, 119, 170, 95, 110, 63, 35, 141, 47, 45, 144, 49, 53, 112, 65, 45, 123, 171, 97, 97, 193, 95, 91, 186, 94, 126, 172, 107, 116, 68, 48, 141, 45, 49, 142, 43, 46, 116, 68, 46, 121, 171, 98, 96, 193, 96, 90, 187, 90, 124, 175, 100, 116, 69, 41, 140, 46, 44, 139, 44, 48, 115, 68, 48, 124, 172, 98, 94, 190, 92, 91, 186, 94, 132, 178, 113, 113, 65, 45, 142, 46, 50, 150, 51, 54, 110, 62, 40, 125, 175, 102, 91, 188, 91, 95, 192, 95, 119, 170, 95, 110, 63, 35, 141, 47, 45, 144, 49, 53, 112, 65, 45, 123, 171, 97, 97, 193, 95, 91, 186, 94, 126, 1

In [143]:
# [133, 56, 46] in accepted_colours

In [144]:
# Let us now find the length of the red lines that represent the sync signal

def find_next_pixel_in_row(pixel, row, accepted_deviations):
    """Returns the column of the next pixel of a given colour
       (with deviations) in a row from a converted to list with RGB 
       colours PNG
       Input: 2-tuple of int, list of int with len%3==0, colourlist
       Output: int (returns -1 specifically if none are found)
    """
    
    l = len(row)
    
    if pixel[1] >= l-1:
        return -1
    
    for i in range(pixel[1]+3, l, 3):
        # if [row[i], row[i+1], row[i+2]] in accepted_deviations:
        if withinDeviation([row[i], row[i+1], row[i+2]], accepted_deviations):
            return i
    
    return -1

line_length = 1
pr = fr[:]
r = (pr[0], find_next_pixel_in_row(pr, img[pr[0]], accepted_colours[:]))
# print(pr, r)
if not(r[1] == pr[1]+3):
    print("Ooops! Something went wrong!")
else:
    line_length += 1
    while (r[1] == pr[1]+3):
        pr = r
        r = (pr[0], find_next_pixel_in_row(pr, img[pr[0]], accepted_colours[:]))
        line_length += 1

print(line_length) # !!!

5


**We found the sync (clock) line length in our graph!**

In [145]:
print("It is", line_length)

It is 5


Now the information transfer signal itself is ~"black", so we need to find the black colour range as well!

In [146]:
# Let's do just that

black = colourlist([[0, 0, 0], [0, 1, 0], [7, 2, 8]])
# black.add_deviations(60) # experimentally it is somewhere around that
# experimentally the max deviation is somewhere around 60
print(black)

[[0, 0, 0], [0, 1, 0], [7, 2, 8]]


The signal we are currently interested in is *Manchester code* __(as per G.E. Thomas)__.

It is a self-clocking signal, but since we *do* have a clock with it - we use it)

Let us find the height of the Manchester signal in our PNG.

In [147]:
fb = find_first_pixel_of_colour(img, black)

In [148]:
signal_height = 1
# if ([img[fb[0]+1][fb[1]], img[fb[0]+1][fb[1]+1], img[fb[0]+1][fb[1]+2]] in black):
if withinDeviation([img[fb[0]+1][fb[1]], img[fb[0]+1][fb[1]+1], img[fb[0]+1][fb[1]+2]], black, 60):
    signal_height += 1
    i = 2
    rows = len(img)
    # while([img[fb[0]+i][fb[1]], img[fb[0]+i][fb[1]+1], img[fb[0]+i][fb[1]+2]] in black):
    while(withinDeviation([img[fb[0]+i][fb[1]], img[fb[0]+i][fb[1]+1], img[fb[0]+i][fb[1]+2]], black)):
        signal_height += 1
        i += 1
        if (i >= rows):
            break
else:
    print("") # TO DO

In [149]:
print(signal_height)

8


In [150]:
cols = len(img[0])
print(cols)

4623


In [151]:
# Let's quickly find the last red line
...

In [165]:
# A-a-and... here is magic!
res = ""
m = 2*line_length*3-2*3
for i in range(fr[1]+5*3, cols-2*3, m):
    fromUP = withinDeviation([img[fb[0]][i-6], img[fb[0]][i-5], img[fb[0]][i-4]], black, 60)
    # print([img[fb[0]][i-6], img[fb[0]][i-5], img[fb[0]][i-4]])
    # toDOWN = withinDeviation([img[fb[0]+signal_height][i+5*3+1], 
    #                           img[fb[0]+signal_height][i+5*3+2], img[fb[0]+signal_height][i+5*3+3]], black, 60)
    cond = fromUP # and toDOWN
    if cond:
        print(1, end=" ")
        res = res + "1"
    else:
        print(0, end=" ")
        res = res + "0"
# print(fr[1]+4*3)
# print(withinDeviation([img[fb[0]][fr[1]+5*3-6], img[fb[0]][fr[1]+5*3-5], img[fb[0]][fr[1]+5*3-4]], black, 60))
# print(withinDeviation([img[fb[0]+signal_height][fr[1]+5*3+1], 
#                              img[fb[0]+signal_height][fr[1]+5*3+2], img[fb[0]+signal_height][fr[1]+5*3+3]], black, 60))

0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 

In [166]:
ans = []
for i in range(0, len(res), 8):
    ans.append(int('0b'+res[i:i+8], 2))
# print(ans)

In [173]:
for i in range(0, len(ans)):
    print(ans[i])

49
49
49
48
49
49
49
49
49
49
49
48
49
49
48
49
49
49
49
48
49
48
49
49


Huzzah!

In [175]:
"""res2d = ""
for i in range(0, len(ans)):
    res2d += chr(ans[i])

ans2d = []
for i in range(0, len(res2d), 8):
    print(int('0b'+res2d[i:i+8], 2))"""

'res2d = ""\nfor i in range(0, len(ans)):\n    res2d += chr(ans[i])\n\nans2d = []\nfor i in range(0, len(res2d), 8):\n    print(int(\'0b\'+res2d[i:i+8], 2))'